In [45]:
import glob
import pandas as pd
import os

import pyspark
import warnings
warnings.filterwarnings(action='ignore')
# from pyspark import SparkContext
# sc = SparkContext()

# from pyspark.sql import SQLContext
# sqlContext = SQLContext(sc)



from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [46]:
os.getcwd()

'/Users/syedmisbah.ahmed/Desktop/Weather Data'

In [47]:
#Read Clicks Data
# files_path = 
files_path = glob.glob( '**/*.csv',recursive=True)

files_path = files_path
print(files_path)
# df_clicks = pd.DataFrame()
# for file in files_path:
#     print(file)
#     df = pd.read_parquet(file)
#     df_clicks = df_clicks.append(df)
# print(df_clicks.shape)

['2020.csv', '2021.csv', '2019.csv', '2018.csv', '2015.csv', '2016.csv', '2017.csv', 'output/weather_data_year_month.csv', 'output/agg_year.csv', 'output copy/weather_data_year_month.csv', 'output copy/agg_year.csv']


In [48]:
from pyspark.sql.types import StructType, StructField, IntegerType,StringType,DoubleType

schema = StructType([
    StructField("STATION", StringType(), True),
    StructField("YEARMONTH", StringType(), True),
    StructField("KPI", StringType(), True),
    StructField("VALUE", DoubleType(), True),
    StructField("FLAG1", StringType(), True),
    StructField("FLAG2", StringType(), True),
    StructField("FLAG3", StringType(), True),
    StructField("FLAG4", StringType(), True)])


schema_less = StructType([
    StructField("STATION", StringType(), True),
    StructField("YEARMONTH", StringType(), True),
    StructField("KPI", StringType(), True),
    StructField("VALUE", DoubleType(), True),
    StructField("COUNTRY_CODE", DoubleType(), True)])

In [49]:
main_df = spark.createDataFrame([],schema = schema_less)
main_df.show()

+-------+---------+---+-----+------------+
|STATION|YEARMONTH|KPI|VALUE|COUNTRY_CODE|
+-------+---------+---+-----+------------+
+-------+---------+---+-----+------------+



In [50]:
col_names = ['STATION','YEARMONTH','KPI','VALUE','COUNTRY_CODE']

In [51]:
files_path

['2020.csv',
 '2021.csv',
 '2019.csv',
 '2018.csv',
 '2015.csv',
 '2016.csv',
 '2017.csv',
 'output/weather_data_year_month.csv',
 'output/agg_year.csv',
 'output copy/weather_data_year_month.csv',
 'output copy/agg_year.csv']

In [52]:
files_path =files_path[0:7]

In [53]:
files_path

['2020.csv',
 '2021.csv',
 '2019.csv',
 '2018.csv',
 '2015.csv',
 '2016.csv',
 '2017.csv']

In [54]:
for i in files_path:
    print(i)
    df = spark.read.csv(i,inferSchema=False,schema = schema)
    df = df.withColumn('COUNTRY_CODE', df.STATION.substr(0,2))
    df = df.filter(df.COUNTRY_CODE == 'US').select(col_names).filter(df.KPI.isin(['PRCP','SNOW','SNWD','TMAX','TAVG','TMIN']))
    main_df = main_df.unionByName(df)

2020.csv
2021.csv
2019.csv
2018.csv
2015.csv
2016.csv
2017.csv


In [55]:
pivotDF = main_df.groupBy("STATION","YEARMONTH").pivot("KPI").sum("VALUE")
# pivotDF.printSchema()

In [56]:
cleaned_df = pivotDF.dropna(subset=['SNOW','PRCP'],how='all')

In [57]:
# from pyspark.sql.functions import col, countDistinct
# pivotDF.agg(countDistinct(col("YEARMONTH")).alias("count")).show()

In [58]:
# pivotDF.write.parquet("output/weather_data.parquet")

In [59]:
stn_df = spark.read.parquet('output/stn.parquet')

In [60]:
# print(pivotDF.count())
join_df = pivotDF.join(stn_df,how='left',on='STATION')
# print(join_df.count())

In [61]:
# join_df.show()

In [62]:
# join_df.filter(join_df.STATE_CITY_NAME.isNull()).count()

In [63]:
join_df = join_df.fillna(subset=['SNOW','PRCP'],value=0.0)

In [64]:
# val = '20160106'
# val.substr(0,2)

In [65]:
join_df = join_df.withColumn("YEAR", join_df.YEARMONTH.substr(0,4)).withColumn("MONTH", join_df.YEARMONTH.substr(5,2)).withColumn("DAY", join_df.YEARMONTH.substr(7,2))

In [66]:
# join_df.show()

In [67]:
import pyspark.sql.functions as F

In [68]:
agg_df = join_df.groupBy("YEAR","MONTH","STATE_CITY_NAME").agg(F.max("SNOW").alias("MAX_SNOW"), \
                                                                      F.min("SNOW").alias("MIN_SNOW"), \
                                                                     F.max("PRCP").alias("MAX_PRCP"), \
                                                                     F.min("PRCP").alias("MIN_PRCP"), \
                                                                     F.mean("SNOW").alias("MEAN_SNOW"), \
                                                                     F.mean("PRCP").alias("MEAN_PRCP"), \
                                                                     F.percentile_approx("SNOW", 0.5).alias("MEDIAN_SNOW"), \
                                                                     F.percentile_approx("PRCP", 0.5).alias("MEDIAN_PRCP"), \
                                                                     F.mean("LONG").alias("MEAN_LONG"), \
                                                                     F.mean("LAT").alias("MEAN_LAT") )

In [69]:
agg_year_only_df = join_df.groupBy("YEAR","STATE_CITY_NAME").agg(F.max("SNOW").alias("MAX_SNOW"), \
                                                                      F.min("SNOW").alias("MIN_SNOW"), \
                                                                     F.max("PRCP").alias("MAX_PRCP"), \
                                                                     F.min("PRCP").alias("MIN_PRCP"), \
                                                                     F.mean("SNOW").alias("MEAN_SNOW"), \
                                                                     F.mean("PRCP").alias("MEAN_PRCP"), \
                                                                     F.percentile_approx("SNOW", 0.5).alias("MEDIAN_SNOW"), \
                                                                     F.percentile_approx("PRCP", 0.5).alias("MEDIAN_PRCP"), \
                                                                     F.mean("LONG").alias("MEAN_LONG"), \
                                                                     F.mean("LAT").alias("MEAN_LAT") )

In [71]:
agg_year_only_df.write.parquet('output/agg_data_year_level.parquet')
agg_df.write.parquet('output/agg_df_y_m.parquet')

In [72]:
join_df.write.parquet('output/full_data.parquet')